https://qiita.com/yuki_uchida/items/09fda4c5c608a9f53d2f#ツイートを文章ベクトルに変換する

https://nlp.ist.i.kyoto-u.ac.jp/index.php?ku_bert_japanese BERT日本語Pretrainedモデル

In [1]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np

In [25]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import numpy as np
model = BertModel.from_pretrained('bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers')
bert_tokenizer = BertTokenizer("bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt",
                               do_lower_case=False, do_basic_tokenize=False)

OSError: bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [ ]:
import pandas as pd
import re

In [ ]:
reviews_df = pd.read_csv("/Users/hoteison/Desktop/arranged_tripadvisor.csv", encoding='utf-8')
reviews_df = reviews_df["body"]
reviews_df

0     東京タワーのトップデッキツアーに参加。やはり冬場、空が澄んでいる時がおすすめ。外国人まだコロ...
1     2018年見た【東京ﾀﾜｰ】が《子供の日》に合わせた企画；《333匹（東京ﾀﾜｰの高さに合わ...
2     150mのメインデッキ、250mのトップデッキの、2つの展望台がある。以前来た時もトップデッ...
3     2度目の訪問で、今回は夜景を見てきました。きっかけは近くに宿泊した際に、オレンジ色にライトア...
4     23区在住ですが、汐留のホテルに泊まりに行ったとき夜のライトアップを散歩で眺め、昼は上ってみ...
5     東京タワー🗼東京って言ったらね…東京タワー🗼って言われたいい時代もありました🗼今はすっかり忘...
6     トップデッキツアーが最悪。ただトップデッキに行かせてくれればいいのに、おもしろくもない説明を...
7     今さら初めての東京タワー。スカイツリーに行ったこともないですが、高さなどで劣る東京タワーはあ...
8     スカイツリーと比べると、高さは見劣りするものの、展示の工夫などがあり、とても楽しめました。タ...
9     東京に8年住んでいるときには一度も行った事がなかった東京タワー。その後大阪に住むようになって...
10    自転車で訪れました。近くに行ってもなかなか探せない。今となっては意外と低いなぁと思います。早...
11    時間があったので、ぶらりと立ち寄り。天気も良くなかったので、メインデッキまでにしました。料金...
12    実際に登ったのは中学生の時でしたが、最近は眺めて楽しんでます。今回は一泊二日の東京の旅でした...
13    昨年１０月に行きました。当日浅草ビューホテルで東京スカイツリーを見て、過去に東京スカイツリー...
14    三十年ぶりくらい？に行きましたが、コロナ禍で人も少なく、ゆったりと展望を楽しめました。しかし...
15    首都圏に住んでいるとなかなか行かない東京タワー。10年ぶりくらいに登ってみました。昔は天井が...
16    コロナ禍で実施された、メインデッキ昇り階段にチャレンジしました。階段を昇りながら見る景色は、...
17    散歩途中に、久しぶりにしたから東京タワーを眺めました。大きいです。鉄骨もしっかりとし

In [ ]:
def compute_vector(text, model, bert_tokenizer, juman_tokenizer):
    use_model = model
    tokens = juman_tokenizer.tokenize(text)
    bert_tokens = bert_tokenizer.tokenize(" ".join(tokens))
    ids = bert_tokenizer.convert_tokens_to_ids(
        ["[CLS]"] + bert_tokens[:126] + ["[SEP]"])
    tokens_tensor = torch.tensor(ids).reshape(1, -1)
    use_model.eval()
    with torch.no_grad():
        all_encoder_layers, _ = use_model(tokens_tensor)
    pooling_layer = -2
    embedding = all_encoder_layers[0][pooling_layer].numpy()
    # embedding = all_encoder_layers[0].numpy()
    # return np.mean(embedding, axis=0)
    return embedding

In [ ]:
reviews_df = pd.read_csv("/Users/hoteison/Desktop/arranged_tripadvisor.csv", encoding='utf-8')
reviews_df["body"] = reviews_df["body"].astype(str) #一応文字列にしておく
reviews_df = reviews_df["body"]

## 文章ベクトル変換後にその結果を格納するための配列と、元のツイートを保存しておくための配列を宣言する
vectors = []
reviews = []
for review in reviews_df:
    review = re.sub('\n', " ", review)  #改行文字のの削除
    strip_review = re.sub(r'[︰-＠]', "", review)  #全角記号の削除
    try:
        if len(strip_review) > 3: #単語数が少なすぎると適切なベクトルが得られない可能性があるため
            vector = compute_vector(
                strip_review, model, bert_tokenizer, juman_tokenizer)
            vectors.append(vector)
            reviews.append(review)
    except Exception as e:
        continue
## 文章ベクトルに変換したものをtsvにおく。(可視化ツールがtsvを要求してくるのでtsvにする)
pd.DataFrame(reviews).to_csv('./reviews_text.tsv', index=False, header=None)
pd.DataFrame(vectors).to_csv('./reviews_vector.tsv', sep='\t', index=False, header=None)